- 导入相关的包

In [1]:
from langchain.llms.base import LLM
from langchain.chains import RetrievalQA
from zhipuai import ZhipuAI
from typing import Dict, Any, Mapping
from pydantic import Field
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional, Dict, Union, Tuple
import json
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyMuPDFLoader
from langchain.memory import ConversationBufferMemory
import matplotlib.pyplot as plt
from langchain.chains import ConversationalRetrievalChain
import matplotlib.image as mpimg
from langchain.text_splitter import RecursiveCharacterTextSplitter
import cv2
import requests
import os
from langchain import PromptTemplate, LLMChain
from langchain.chains import LLMRequestsChain
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain.callbacks.manager import CallbackManagerForLLMRun
from __future__ import annotations
import logging
from typing import Any, Dict, List, Optional
from langchain.embeddings.base import Embeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.pydantic_v1 import BaseModel, root_validator
from langchain.utils import get_from_dict_or_env
from paddleocr import PaddleOCR
logger = logging.getLogger(__name__)

- 自定义相关的LLM类：定义了一个名为 Self_LLM 的自定义 LLM 类，该类继承自 LLM。类包含访问 URL、模型名称、请求超时、温度系数、API 密钥和额外参数。提供了一个方法返回默认调用参数，包括温度和请求超时，并将这些参数与模型参数合并。另一个方法返回识别参数，将模型名称与默认参数合并。这个类用于灵活配置和访问自定义 LLM 模型。

In [2]:
class Self_LLM(LLM):
    # 自定义 LLM
    url: str = "https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={api_key}&client_secret={secret_key}"
    model_name: str = "gpt-3.5-turbo"
    # 访问时延上限
    request_timeout: float = None
    # 温度系数
    temperature: float = 0.1
    api_key: str = None
    model_kwargs: Dict[str, Any] = Field(default_factory=dict)

    # 定义一个返回默认参数的方法
    @property
    def _default_params(self) -> Dict[str, Any]:
        """获取调用默认参数。"""
        normal_params = {
            "temperature": self.temperature,
            "request_timeout": self.request_timeout,
        }
        # print(type(self.model_kwargs))
        return {**normal_params}

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {**{"model_name": self.model_name}, **self._default_params}

- 自定义相关的文心大模型：通过相关的apikey和secretkey得到对应的access token，并且通过access_token向文心一言的API地址发起请求，最终得到相应的服务。

In [3]:
def get_access_token(api_key: str, secret_key: str):

    url = f"https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={api_key}&client_secret={secret_key}"
    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json().get("access_token")


class Wenxin_LLM(Self_LLM):
    # 文心大模型的自定义 LLM
    url: str = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/ernie-4.0-turbo-8k?access_token={}"
    # Secret_Key
    secret_key: str = None
    # access_token
    access_token: str = None

    def init_access_token(self):
        if self.api_key != None and self.secret_key != None:
            try:
                self.access_token = get_access_token(self.api_key, self.secret_key)
            except Exception as e:
                print(e)
                print("获取 access_token 失败，请检查 Key")
        else:
            print("API_Key 或 Secret_Key 为空，请检查 Key")

    def _call(self, prompt: str, stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs: Any):
        # 如果 access_token 为空，初始化 access_token
        if self.access_token == None:
            self.init_access_token()
        # API 调用 url
        url = self.url.format(self.access_token)
        # 配置 POST 参数
        payload = json.dumps({
            "messages": [
                {
                    "role": "user",  # user prompt
                    "content": "{}".format(prompt)  # 输入的 prompt
                }
            ],
            'temperature': self.temperature
        })
        headers = {
            'Content-Type': 'application/json'
        }
        # 发起请求
        response = requests.request("POST", url, headers=headers, data=payload, timeout=self.request_timeout)
        if response.status_code == 200:
            js = json.loads(response.text)
            print(js)
            return js["result"]
        else:
            return "请求失败"

    @property
    def _llm_type(self) -> str:
        return "Wenxin"

#### 测试大模型api是否可用

In [4]:
from dotenv import find_dotenv, load_dotenv
import os

_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
wenxin_api_key = os.environ["wenxin_api_key"]
wenxin_secret_key = os.environ["wenxin_secret_key"]

In [5]:
llm = Wenxin_LLM(api_key=wenxin_api_key, secret_key=wenxin_secret_key)
llm("你好")

{'id': 'as-ibbafhmyq2', 'object': 'chat.completion', 'created': 1746586721, 'result': '你好，我是百度公司研发的知识增强大语言模型，我的中文名是文心一言，英文名是ERNIE Bot，很高兴认识你！我可以为你解答问题、创作文本、进行知识推理，如果你有需要的话，我还可以跟你一起聊天、分享笑话或者讲故事。', 'is_truncated': False, 'need_clear_history': False, 'finish_reason': 'normal', 'usage': {'prompt_tokens': 1, 'completion_tokens': 51, 'total_tokens': 52}}


'你好，我是百度公司研发的知识增强大语言模型，我的中文名是文心一言，英文名是ERNIE Bot，很高兴认识你！我可以为你解答问题、创作文本、进行知识推理，如果你有需要的话，我还可以跟你一起聊天、分享笑话或者讲故事。'

- 自定义相关的Zhipu的Embedding类：通过利用Zhipu Embedding进行相关的嵌入操作，从而实现后续将已有知识存入知识库的操作，这里的定义方法跟上述的文心一言大模型相似，所以这里就不再过多的赘述。

In [6]:
class ZhipuAIEmbeddings(BaseModel, Embeddings):
    """`Zhipuai Embeddings` embedding models."""

    client: Any
    """`zhipuai.ZhipuAI"""

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """
        实例化ZhipuAI为values["client"]

        Args:

            values (Dict): 包含配置信息的字典，必须包含 client 的字段.
        Returns:

            values (Dict): 包含配置信息的字典。如果环境中有zhipuai库，则将返回实例化的ZhipuAI类；否则将报错 'ModuleNotFoundError: No module named 'zhipuai''.
        """
        from zhipuai import ZhipuAI
        from dotenv import find_dotenv, load_dotenv
        import os
        zhipu_api_key = os.environ["zhipu_api_key"]

        values["client"] = ZhipuAI(api_key=zhipu_api_key)
        return values

    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.

        Args:
            texts (str): 要生成 embedding 的文本.

        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表.
        """
        embeddings = self.client.embeddings.create(
            model="embedding-2",
            input=text
        )
        # 单个文本的 embedding 是一个列表，包含一个字典。每个字典都有一个 'embedding' 的键，其值是浮点数列表.
        # 因此，我们从列表中取出第一个元素（字典），然后从这个字典中获取 'embedding' 的值.
        return embeddings.data[0].embedding

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embedding.
        Args:
            texts (List[str]): 要生成 embedding 的文本列表.

        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表。每个 embedding 都表示为一个浮点值列表。
        """
        return [self.embed_query(text) for text in texts]

- 加载相关的pdf文档，然后进行相关的文档切分，这里主要是针对我们收集到的pdf文档进行相关的切分操作，从而实现了相关的数据预处理的操作

In [7]:
from tqdm import tqdm  # 显示进度条
from typing import List

directory = './others/data/'
# 加载 PDF
loaders_chinese = []
for root, dirs, files in os.walk(directory):
    for file in files:
        file_path = os.path.join(root, file)
        loaders_chinese.append(PyMuPDFLoader(file_path))
docs = []
for loader in loaders_chinese:
    docs.extend(loader.load())
# 切分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
split_docs = text_splitter.split_documents(docs)
embedding = ZhipuAIEmbeddings()

In [8]:
print(len(split_docs))

301


- 可持久化的地址

In [9]:
persist_directory = './others/vector_db/chroma'

- 删除原来的向量数据库

In [10]:
# linux 下删除文件夹
# !rm -rf './others/vector_db/chroma'
# windows 下删除文件夹
!rmdir /s /q "./others/vector_db/chroma"

- 建立相关的向量数据库：利用Chroma相关的函数和对应的Embedding类进行建立相关的向量数据库

In [11]:
vectordb = Chroma.from_documents(
    documents=split_docs[:1000],
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

- 统计向量数据库中存储的数量

In [12]:
vectordb.persist()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：301


In [13]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

- 建立相关的基于文心一言大模型的LLM，**RAG**技术的核心代码，并且我模型采用了Prompt和记忆操作，并且通过强大的向量数据库的方式进行增强检索。其中prompt操作，是通过反复的迭代和验证得到的。

In [14]:
from dotenv import find_dotenv, load_dotenv
import os

_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
wenxin_api_key = os.environ["wenxin_api_key"]
wenxin_secret_key = os.environ["wenxin_secret_key"]

template = """
你现在是一名由小陈研发的虚拟医生，具备丰富的医学知识和临床经验。
你只能根据以下知识内容回答问题，不允许编造。如果无法回答，就回答“知识库中没有找到相关信息”。不要回答与医学知识无关的内容。
知识内容如下: {context}，现在你需要根据这段知识内容，结合你的医学知识，为患者提供医疗建议。尽量回答的准确和详细，
来为这个患者解答以下问题。
问题：{question}"""
memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)
prompt_with_context  = PromptTemplate(input_variables=["context", "question"],template=template)
print(prompt_with_context )
llm_wenxin = Wenxin_LLM(api_key=wenxin_api_key, secret_key=wenxin_secret_key)
llm = RetrievalQA.from_chain_type(
            llm_wenxin,
            retriever=vectordb.as_retriever(),
            memory=memory,
            chain_type_kwargs = {"prompt":prompt_with_context }
            )

input_variables=['context', 'question'] template='\n你现在是一名由小陈研发的虚拟医生，具备丰富的医学知识和临床经验。\n你只能根据以下知识内容回答问题，不允许编造。如果无法回答，就回答“知识库中没有找到相关信息”。不要回答与医学知识无关的内容。\n知识内容如下: {context}，现在你需要根据这段知识内容，结合你的医学知识，为患者提供医疗建议。尽量回答的准确和详细，\n来为这个患者解答以下问题。\n问题：{question}'


In [15]:
template_without_context = """
你现在是一名由小陈研发的博学的虚拟医生，具备丰富的医学知识和临床经验。
当用户提问除医疗外的问题时，你也能给出准确的回答。
请结合你自身的知识，准确、详细地回答以下问题：
问题：{question}"""
memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)
prompt_without_context  = PromptTemplate(input_variables=["question"],template=template_without_context)
print(prompt_without_context)
# 无知识库链：LLMChain
llm_without_kd = LLMChain(
    llm=llm_wenxin,
    prompt=PromptTemplate(input_variables=["question"], template=template_without_context),
    memory=memory
)

input_variables=['question'] template='\n你现在是一名由小陈研发的博学的虚拟医生，具备丰富的医学知识和临床经验。\n当用户提问除医疗外的问题时，你也能给出准确的回答。\n请结合你自身的知识，准确、详细地回答以下问题：\n问题：{question}'


- 定义相关的tools（API和小模型），从而能够构建出相应的Agent，这里主要是构建药物API和OCR识别的tool，以便大模型使用时候调用

In [16]:
Chain = Wenxin_LLM(api_key=wenxin_api_key, secret_key=wenxin_secret_key)  
 
def search_medicine(question: str, medicines: list[str]) -> dict:
    """
    根据给定的药品名称查询药品说明书，仅用于药品相关问题处理。
    不支持与药品无关的任务，如天气、旅游、日常信息查询。
    """
    print(f"尝试查询药品列表：{medicines}")

    from dotenv import find_dotenv, load_dotenv
    import os, requests
    tian_api_key = os.environ["tian_api_key"]

    for med in medicines:
        url = f"https://apis.tianapi.com/yaopin/index?key={tian_api_key}&word={med}"
        response = requests.get(url)
        print(response.json())

        if response.status_code == 200 and response.json().get("code") == 200:
            requests_result = response.json()['result']['list'][0]['content']
            prompt_template = """以下是药物'{medicine}'的基本信息：
            >>> {requests_result} <<<
            根据以上基本信息，回答以下这个问题：
            >>> {question} <<<"""
            prompt = PromptTemplate(
                input_variables=["question", "medicine", "requests_result"],
                template=prompt_template
            )
            # print(prompt.input_variables)
            chain = LLMChain(llm=Chain, prompt=prompt)
            inputs = {
                "question": question,
                "medicine": med,
                "requests_result": requests_result
            }
            output = chain.invoke(inputs)
            print(f"这是识别药物输出：{output}")
            print(type(output))
            return output

    return {
        "output": "很抱歉，未能查到该药品的说明书信息。"
    }

def ocr_medicine(question: str, path: str) -> dict:
    """
    此函数只用于从药品包装图片中提取文字，用于识别药品的商品名和通用名。
    仅适用于药品识别任务，不应用于天气、新闻或其他非药品类问题。
    """

    prompt_template = """根据以下药品包装识别的文字内容，提取药品名称：
    >>> {describtion} <<<
    请提取商品名和通用名，并用逗号分隔。不要包含'片','丸','胶囊'等后缀。
    """
    prompt = PromptTemplate(
        input_variables=["describtion"],
        template=prompt_template
    )
    chain = LLMChain(llm=Chain, prompt=prompt)

    ocr = PaddleOCR(use_angle_cls=True, lang="ch")
    img = cv2.imread(path)
    describtion = ocr.ocr(img)
    result = []
    for line in describtion:
        line_text = ' '.join([word_info[-1][0] for word_info in line])
        result.append(line_text)
    ocr_text = ' '.join(result)

    output = chain.invoke({"describtion": ocr_text})
    # print(f"这是输出：{output}")
    output = output['text']
    print(f"这是图像识别输出：{output}")
    # 解析大模型返回的字符串为列表
    names = [x.strip() for x in output.split(",") if x.strip()]

    return {
        "text": output,
        "medicine_candidates": names
    }
functions=[convert_to_openai_tool(search_medicine),convert_to_openai_tool(ocr_medicine)]
print(functions)

[{'type': 'function', 'function': {'name': 'search_medicine', 'description': '根据给定的药品名称查询药品说明书，仅用于药品相关问题处理。\n不支持与药品无关的任务，如天气、旅游、日常信息查询。', 'parameters': {'type': 'object', 'properties': {'question': {'type': 'string'}, 'medicines': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['question', 'medicines']}}}, {'type': 'function', 'function': {'name': 'ocr_medicine', 'description': '此函数只用于从药品包装图片中提取文字，用于识别药品的商品名和通用名。\n仅适用于药品识别任务，不应用于天气、新闻或其他非药品类问题。', 'parameters': {'type': 'object', 'properties': {'question': {'type': 'string'}, 'path': {'type': 'string'}}, 'required': ['question', 'path']}}}]


In [20]:
zhipu_api_key = os.environ["zhipu_api_key"]
Chat = ZhipuAI(api_key=zhipu_api_key)

def get_response(message, use_knowledge=True):
    result = Chat.chat.completions.create(model="glm-4",messages=[{"role": "user","content": message}],
                               tools=functions)
    if(result.choices[0].message.tool_calls == None):
         mmr_docs = vectordb.max_marginal_relevance_search(message,k=3)
         input_data = {
            'input_documents': mmr_docs,
            'question': message,
        }
         if use_knowledge:
            print("使用知识库回答")
            return llm({"query": message})["result"]
         else:
           print("不使用知识库回答")
           return llm_without_kd.run({"question": message})
    else:
        tool_call = result.choices[0].message.tool_calls[0]
        args = tool_call.function.arguments
        if tool_call.function.name == "search_medicine":
            print("test1")
            function_result = search_medicine(**json.loads(args))
            return function_result['text']
        if tool_call.function.name == "ocr_medicine":
            print("test2")
            function_result = ocr_medicine(**json.loads(args))
            return function_result  # 不只返回 text，还返回药品名数组

In [21]:
A = get_response("什么是南瓜书")
print(A)

TypeError: Completions.create() got an unexpected keyword argument 'function_call'

In [ ]:
path = './others/xisimin.jpg'
img = mpimg.imread(path)
plt.imshow(img)
plt.axis('off')
plt.show()
A = get_response(f"这是药盒的文件路径{path}，该药的名称是什么？")
print(A)
B = get_response(f"{A},请问它的药品说明书？")
print(B)

- 创建Web界面

In [ ]:
import gradio as gr

def format_chat_prompt(message, chat_history):
    prompt = ""
    for turn in chat_history:
        user_message, bot_message = turn
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

def respond(file, message, chat_history, kd_mode):
    error_msg = ""
    try:
        # 默认不清空文件
        file_update = gr.update()

        if file is not None:
            # Gradio 上传的文件对象是字典，真实路径在 file.name
            file_path = file.name  # 获取临时文件路径
            img = cv2.imread(file_path)
            if img is None:
                raise ValueError("无法读取图片，请检查文件路径或格式")
            
            # 调用处理函数时传递真实路径
            medicine = get_response(f"这是药盒的文件路径 {file_path}，该药的名称是什么？")
            bot_message = get_response(f"{medicine}, {message}？")
            chat_history.append((message, bot_message))

            # 成功处理后清空文件上传区
            file_update = gr.update(value=None)

        else:
            print(f"当前选项：{kd_mode}")
            bot_message = get_response(message, use_knowledge=(kd_mode == "选择知识库回答"))
            chat_history.append((message, bot_message))

        return "", chat_history,  gr.update(visible=False), file_update

    except Exception as e:
        # 打印详细错误日志
        import traceback
        traceback.print_exc()
        error_msg = f"⚠️ 请求失败：{str(e)}"
    return message, chat_history, gr.update(value=error_msg, visible=True), gr.update()

with gr.Blocks(
    theme=gr.themes.Soft(
        primary_hue="emerald",
        font=[gr.themes.GoogleFont("Noto Sans SC")]
    ),
    css=".error-banner {background: #fff3f3!important; border: 1px solid #ffb3b3!important;}"
) as demo:
    
    # 错误提示横幅
    error_banner = gr.HTML(
        visible=False,
        elem_classes="error-banner",
        elem_id="error_banner"
    )
    
    # 标题
    gr.Markdown("""
    <div style="text-align: center;">
        <h1 style="color: #2e7d32;">药品信息咨询助手</h1>
        <p style="color: #666;">上传药盒图片或直接提问获取药品信息</p>
    </div>
    """)
    
    with gr.Row(equal_height=False):
        # 左侧上传区
        with gr.Column(scale=1, min_width=280):
            file = gr.File(
                label='📁 上传图片',
                file_types=['.jpg','.png'],
                height=200,
                elem_classes="box-panel",
                file_count="single"
            )
            gr.Markdown("支持格式：JPG/PNG")
        
            # 新增：选择知识库模式
            kd_mode = gr.Dropdown(
                choices=["选择知识库回答", "不选择知识库回答"],
                value="选择知识库回答",
                label="知识库模式",
                interactive=True,
                elem_classes="kd-dropdown"
            )

        # 右侧聊天区
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                bubble_full_width=False,
                avatar_images=(
                    "./user.png",
                    "./bot.png" 
                ),
                height=500,
                show_label=False,
                elem_classes="chat-container"
            )
            
            # 输入区域
            with gr.Row(elem_classes="input-group"):
                msg = gr.Textbox(
                    show_label=False,
                    placeholder="输入您的问题...",
                    container=True,
                    max_lines=3,
                    scale=8
                )
                btn = gr.Button("发送", variant="primary", scale=1)
    
    # 操作栏
    with gr.Row():
        clear = gr.ClearButton(
            components=[msg, chatbot, file],
            value="🧹 清空对话",
            elem_classes="clear-btn"
        )
    
    # 交互事件
    btn.click(
        respond,
        inputs=[file, msg, chatbot, kd_mode],
        outputs=[msg, chatbot, error_banner, file]
    )
    # 新增：提交事件，支持回车发送消息
    msg.submit(
        respond,
        inputs=[file, msg, chatbot, kd_mode],
        outputs=[msg, chatbot, error_banner, file]
    )

# 自定义CSS（可单独保存为styles.css）
custom_css = """
.box-panel {
    border: 1px solid #e0e0e0 !important;
    border-radius: 12px !important;
    padding: 20px !important;
    background: #f8fff8 !important;
}

.chat-container {
    border-radius: 16px !important;
    box-shadow: 0 4px 12px rgba(0,0,0,0.1) !important;
    padding: 20px !important;
}

.input-group {
    border: 1px solid #e0e0e0 !important;
    border-radius: 24px !important;
    padding: 12px 16px !important;
    background: white !important;
}

/* 让 Dropdown 向下弹出，解决“向上弹”问题 */
.kb-dropdown .wrap.svelte-1ipelgc {
    position: relative !important;
    z-index: 9999 !important;
}
.kb-dropdown .options {
    top: 100% !important;
    bottom: auto !important;
}

.clear-btn {
    margin-top: 15px !important;
}

#error_banner {
    width: 100%;
    padding: 12px 20px;
    border-radius: 8px;
    margin: 10px 0;
    color: #d32f2f;
    font-size: 14px;
}
"""

demo.css = custom_css

if __name__ == "__main__":
    gr.close_all()
    demo.launch()

### 页面布局测试

In [21]:
# import gradio as gr

# with gr.Blocks(
#     theme=gr.themes.Soft(
#         primary_hue="emerald",
#         font=[gr.themes.GoogleFont("Noto Sans SC")]
#     ),
#     css=".error-banner {background: #fff3f3!important; border: 1px solid #ffb3b3!important;}"
# ) as demo:
    
#     # 错误提示横幅
#     error_banner = gr.HTML(
#         visible=False,
#         elem_classes="error-banner",
#         elem_id="error_banner"
#     )
    
#     # 标题
#     gr.Markdown("""
#     <div style="text-align: center;">
#         <h1 style="color: #2e7d32;">药品信息咨询助手</h1>
#         <p style="color: #666;">上传药盒图片或直接提问获取药品信息</p>
#     </div>
#     """)
    
#     with gr.Row(equal_height=False):
#         # 左侧上传区
#         with gr.Column(scale=1, min_width=280):
#             file = gr.File(
#                 label='📁 上传图片',
#                 file_types=['.jpg','.png'],
#                 height=200,
#                 elem_classes="box-panel"
#             )
#             gr.Markdown("支持格式：JPG/PNG")
        
#             # 新增：选择知识库模式
#             kd_mode = gr.Dropdown(
#                 choices=["选择知识库回答", "不选择知识库回答"],
#                 value="选择知识库回答",
#                 label="知识库模式",
#                 interactive=True,
#                 elem_classes="kd-dropdown"
#             )

#         # 右侧聊天区
#         with gr.Column(scale=3):
#             chatbot = gr.Chatbot(
#                 bubble_full_width=False,
#                 avatar_images=(
#                     "user.png",
#                     "bot.png" 
#                 ),
#                 height=500,
#                 show_label=False,
#                 elem_classes="chat-container"
#             )
            
#             # 输入区域
#             with gr.Row(elem_classes="input-group"):
#                 msg = gr.Textbox(
#                     show_label=False,
#                     placeholder="输入您的问题...",
#                     container=True,
#                     max_lines=3,
#                     scale=8
#                 )
#                 btn = gr.Button("发送", variant="primary", scale=1)
    
#     # 操作栏
#     with gr.Row():
#         clear = gr.ClearButton(
#             components=[msg, chatbot, file],
#             value="🧹 清空对话",
#             elem_classes="clear-btn"
#         )
    
#     # # 交互事件
#     # btn.click(
#     #     respond,
#     #     inputs=[file, msg, chatbot],
#     #     outputs=[msg, chatbot, error_banner]
#     # )
#     # msg.submit(
#     #     respond,
#     #     inputs=[file, msg, chatbot],
#     #     outputs=[msg, chatbot, error_banner]
#     # )

# # 自定义CSS（可单独保存为styles.css）
# custom_css = """
# .box-panel {
#     border: 1px solid #e0e0e0 !important;
#     border-radius: 12px !important;
#     padding: 20px !important;
#     background: #f8fff8 !important;
# }

# .chat-container {
#     border-radius: 16px !important;
#     box-shadow: 0 4px 12px rgba(0,0,0,0.1) !important;
#     padding: 20px !important;
# }

# .input-group {
#     border: 1px solid #e0e0e0 !important;
#     border-radius: 24px !important;
#     padding: 12px 16px !important;
#     background: white !important;
# }

# /* 让 Dropdown 向下弹出，解决“向上弹”问题 */
# .kb-dropdown .wrap.svelte-1ipelgc {
#     position: relative !important;
#     z-index: 9999 !important;
# }
# .kb-dropdown .options {
#     top: 100% !important;
#     bottom: auto !important;
# }

# .clear-btn {
#     margin-top: 15px !important;
# }

# #error_banner {
#     width: 100%;
#     padding: 12px 20px;
#     border-radius: 8px;
#     margin: 10px 0;
#     color: #d32f2f;
#     font-size: 14px;
# }
# """

# demo.css = custom_css

# if __name__ == "__main__":
#     gr.close_all()
#     demo.launch()

### 百度文本识别测试

In [ ]:
# ocr = PaddleOCR(use_angle_cls=True, lang="ch")
# path = "./others/buluofen.jpg"
# img = cv2.imread(path)
# # 添加以下检查
# if img is None:
#     raise ValueError("图像加载失败！请检查路径是否正确")
# else:
#     print(f"图像形状：{img.shape}, 数据类型：{img.dtype}")

# describtion = ocr.ocr(img)
# result = []
# print(f"这是测试：{describtion}")
# for line in describtion:
#     line_text = ' '.join([word_info[-1][0] for word_info in line])
#     print(line)
#     result.append(line_text)